## Question 1

There are n cities numbered from 1 to n. You are given an array edges of size n-1, where edges[i] = [ui, vi] represents a bidirectional edge between cities ui and vi. There exists a unique path between each pair of cities. In other words, the cities form a tree.

A subtree is a subset of cities where every city is reachable from every other city in the subset, where the path between each pair passes through only the cities from the subset. Two subtrees are different if there is a city in one subtree that is not present in the other.

For each d from 1 to n-1, find the number of subtrees in which the maximum distance between any two cities in the subtree is equal to d.

Return an array of size n-1 where the dth element (1-indexed) is the number of subtrees in which the maximum distance between any two cities is equal to d.

Notice that the distance between the two cities is the number of edges in the path between them.

In [ ]:
class Solution:
    def countSubgraphsForEachDiameter(self, n: int, edges: List[List[int]]) -> List[int]:
        res = [0]*(n-1)
        g = defaultdict(list)
        gs = [0]*n
        dp = [0]*(1<<n)
        for a, b in edges:
            a -= 1
            b -= 1
            g[a].append(b)
            g[b].append(a)
            gs[a] |= (1<<b)
            gs[b] |= (1<<a)
            dp[(1<<a)|(1<<b)] = 1
        dis = [[0]*n for _ in range(n)]
        def dfs(root, cur, depth, parent):
            dis[root][cur] = depth
            for nxt in g[cur]:
                if nxt == parent:
                    continue
                dfs(root, nxt, depth+1, cur)
            return

        for i in range(n):
            dfs(i,i,0,-1)
            
#         print(dis)
#         print(ees)
        
        for state in range(1<<n):
            # print(state, dp[state])
            if dp[state] > 0:
                res[dp[state]-1] += 1
                for i in range(n):
                    if dp[state | (1<<i)] == 0 and (state & gs[i]):
                        dp[state | (1<<i)] = max(dp[state], max(dis[i][j] for j in range(n) if state&(1<<j)))
                    # if dp[state | (1<<i)] == 0 and any((i,j) in ees for j in range(n) if state&(1<<j)):
                    #     dp[state | (1<<i)] = max(dp[state], max(dis[i][j] for j in range(n) if state&(1<<j)))
                    # if state & (1<<i) == 0 and any((i,j) in ees for j in range(n) if state&(1<<j)):
                        # dp[state ^ (1<<i)] = max(dp[state ^ (1<<i)], dp[state], max(dis[i][j] for j in range(n) if state&(1<<j)))
        return res

## Question 2

You are given an m x n binary grid grid where 1 represents land and 0 represents water. An island is a maximal 4-directionally (horizontal or vertical) connected group of 1's.

The grid is said to be connected if we have exactly one island, otherwise is said disconnected.

In one day, we are allowed to change any single land cell (1) into a water cell (0).

Return the minimum number of days to disconnect the grid.

In [ ]:
class Solution:

    def depth_first_search(self , current_i , current_j , R , C , binary_matrix , visited_set):

        visited_set.add((current_i , current_j))

        for next_i , next_j in [[-1 , 0] , [1 , 0] , [0 , -1] , [0 , 1]]:
            
            new_i = (current_i + next_i)

            new_j = (current_j + next_j)

            if((0 <= new_i < R) and (0 <= new_j < C) and ((new_i , new_j) not in visited_set) and (binary_matrix[new_i][new_j] == 1)):
                self.depth_first_search(new_i , new_j , R , C , binary_matrix , visited_set)

    def count_islands(self , R , C , binary_matrix):

        count_of_islands = 0

        visited_set = set()

        for i in range(0 , R):
            
            for j in range(0 , C):
                
                if(((i , j) not in visited_set) and (binary_matrix[i][j] == 1)):

                    count_of_islands += 1

                    self.depth_first_search(i , j , R , C , binary_matrix , visited_set)
        
        return count_of_islands
    
    def minDays(self , binary_matrix : List[List[int]]) -> int:

        R = len(binary_matrix)

        C = len(binary_matrix[0])

        islands_count = self.count_islands(R , C , binary_matrix)

        if(islands_count > 1):
            return 0
        
        count_of_1_s = 0
        
        for i in range(0 , R):

            for j in range(0 , C):

                if(binary_matrix[i][j] == 1):

                    count_of_1_s += 1

                    binary_matrix[i][j] = 0

                    islands_count = self.count_islands(R , C , binary_matrix)

                    if(islands_count > 1):
                        return 1

                    binary_matrix[i][j] = 1

        return (1) if(count_of_1_s == 1) else ((0) if(count_of_1_s == 0) else (2))

## Question 3

You are building a string s of length n one character at a time, prepending each new character to the front of the string. The strings are labeled from 1 to n, where the string with length i is labeled si.

1. For example, for s = "abaca", s1 == "a", s2 == "ca", s3 == "aca", etc.

The score of si is the length of the longest common prefix between si and sn (Note that s == sn).

Given the final string s, return the sum of the score of every si.

In [ ]:
class Solution:
    def sumScores(self, s: str) -> int:
        mod = 119_218_851_371
        hs = 0 
        vals = [0]
        for i, ch in enumerate(s): 
            hs = (hs * 26 + ord(ch) - 97) % mod
            vals.append(hs)
        
        p26 = [1]
        for _ in range(len(s)): p26.append(p26[-1] * 26 % mod)
        
        ans = 0 
        for i in range(len(s)): 
            if s[0] == s[i]: 
                lo, hi = i, len(s)
                while lo < hi: 
                    mid = lo + hi + 1 >> 1
                    hs = (vals[mid] - vals[i]*p26[mid-i]) % mod
                    if hs == vals[mid-i]: lo = mid
                    else: hi = mid - 1
                ans += lo - i 
        return ans 